In [1]:
from sklearn.linear_model import SGDClassifier
import numpy as np
from simulation_util import client_update
import warnings

# hide the warning message temporarily
warnings.simplefilter("ignore")

## Client Update Example

In [2]:
# this data will be provided by the server
features = [[1, 4, 3], [0, 2, 2], [1, 4, 0], [0, 5, 3], [1, 2, 1], [0, 2, 9]]
labels = [1, 0, 1, 0, 1, 0]
weights = [29., 0., 0., -9]
epochs = 3
batch_size = 3

new_weights = client_update(weights, epochs, batch_size, features, labels)
print(new_weights)

[28.48292577  0.          0.         -9.        ]


## Server Update Example

In [3]:
import numpy as np
from simulation_util import server_update

init_weights = [0, 0, 0, 0]
client_fraction = 0.5
num_rounds = 10
epoch = 10
batch_size = 25
display_weight_per_round = True

num_client = 100
samples_per_client = 100
num_features = 3
features = np.random.randint(10, size=(num_client, samples_per_client, num_features))
labels = np.random.randint(2, size=(num_client, samples_per_client))

new_clf = server_update(init_weights, client_fraction, num_rounds, features, labels, epoch, batch_size,display_weight_per_round)

[-5.27111753 -6.03344024 -1.26451517  5.21284141]
[-10.40561087  -2.73003531   3.42485229   3.74318286]
[-18.47873494  -2.71703106   3.14291944   4.18521965]
[ 4.29090589 -2.76024704  6.6403375   5.77258637]
[-2.92563544 -3.48971157  5.95942579  4.66671478]
[-5.89886331 -2.25425103  2.85204615  5.67661476]
[-12.00879624   4.71881083   3.3154799   11.21398267]
[-0.90618671 -3.53589611 -2.99410999 11.20828084]
[-6.40817038e+00  5.97501986e-03 -3.68219429e+00  7.45358790e+00]
[-7.31321881  4.35902824  5.29793986 10.22542928]


## Simulation Runner

In [18]:
from sklearn.model_selection import ParameterGrid, train_test_split
import numpy as np

# Load the data
num_client = 100
samples_per_client = 30
num_features = 3
features = np.random.randint(10, size=(num_client, samples_per_client, num_features))
labels = np.random.randint(2, size=(num_client, samples_per_client))

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.4, random_state=0)

# Find all the permutations of the parameters
param_grid = {"client_fraction": [1, 0.1],
              "epoch": [1, 5],
              "batch_size": [10, samples_per_client],
              "init_weight": [[0,0,0,0]],
              "num_rounds": [10]}

# run training/testing over all parameter combinations to get the best combination
for params in ParameterGrid(param_grid):
    print("Training...")
    classifier = server_update(params["init_weight"], params["client_fraction"], params["num_rounds"], X_train, y_train, params["epoch"], params["batch_size"], False)
    weights = np.append(classifier.coef_[0], classifier.intercept_)

    # need to remove the client dimension from our data for testing 
    # ex: [[[1, 1], [2, 2]], [[3, 3], [4, 4]]] needs to become [[1, 1], [2, 2], [3, 3], [4, 4]] for features 
    # and [[1, 2], [3, 4]] needs to become [1, 2, 3, 4] for labels 
    reshaped_X_test = np.reshape(X_test, (X_test.shape[0] * X_test.shape[1], X_test.shape[2]))
    reshaped_y_test = np.reshape(y_test, y_test.size)
    
    score = classifier.score(reshaped_X_test, reshaped_y_test)

    print('Params: {}\nWeights: {}\nScore: {:f}\n\n'.format(params, weights, score))

Training...
Params: {'batch_size': 10, 'client_fraction': 1, 'epoch': 1, 'init_weight': [0, 0, 0, 0], 'num_rounds': 10}
Weights: [-8.59111289  1.00285832  0.62804378  9.65258255]
Score: 0.514167


Training...
Params: {'batch_size': 10, 'client_fraction': 1, 'epoch': 5, 'init_weight': [0, 0, 0, 0], 'num_rounds': 10}
Weights: [-35.72637181   2.37731063  -6.97718222  19.12348851]
Score: 0.516667


Training...
Params: {'batch_size': 10, 'client_fraction': 0.1, 'epoch': 1, 'init_weight': [0, 0, 0, 0], 'num_rounds': 10}
Weights: [-11.10606145  13.24916061   4.28436622   5.8812667 ]
Score: 0.495000


Training...
Params: {'batch_size': 10, 'client_fraction': 0.1, 'epoch': 5, 'init_weight': [0, 0, 0, 0], 'num_rounds': 10}
Weights: [-17.09368224   6.56114505  -5.77294448  30.82105817]
Score: 0.522500


Training...
Params: {'batch_size': 30, 'client_fraction': 1, 'epoch': 1, 'init_weight': [0, 0, 0, 0], 'num_rounds': 10}
Weights: [-15.41777389   3.33090215   8.43906073  11.31690942]
Score: 0.5041